In [1]:
import recruit_utils
import pandas as pd
import numpy as np

In [2]:
# Import data into pandas data frames
data, stores = recruit_utils.import_data()

recruit_utils.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['tra']['visitor_diff'][mask] = np.nan
recruit_utils.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['tra']['log_visitor_diff'][mask] = np.nan


In [3]:
train, test = recruit_utils.create_train_test(data, stores, clean=True)

In [4]:
train.shape

(250468, 60)